In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [6]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [9]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [11]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())  # 맨 앞자리를 batchsize로 하는듯 -> 채널이 없이 w, h 만 있어도 무관

torch.Size([3, 784])


In [12]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"Before ReLU: {hidden1}\n\n")

Before ReLU: tensor([[ 0.0836,  0.3397, -0.2503, -0.0974, -0.0299,  0.3732, -0.3390,  1.0119,
         -0.2454, -0.1198, -1.2457,  0.4261,  0.4022, -0.1980, -0.5420,  0.0669,
          0.1184, -0.4781,  0.1072,  0.2840],
        [ 0.2232,  0.4901, -0.2789, -0.1164,  0.1000,  0.2607, -0.8644,  1.2661,
         -0.1357, -0.3086, -0.8552,  0.6580,  0.0971,  0.0167, -0.1136, -0.1596,
         -0.0208, -0.0949,  0.3215,  0.1813],
        [ 0.1498,  0.4616,  0.0043, -0.0150,  0.2037,  0.4965, -0.6447,  1.0239,
         -0.1792, -0.1069, -0.7479,  0.4192,  0.1643,  0.3514, -0.2075, -0.0682,
          0.0318,  0.0678,  0.3538,  0.3250]], grad_fn=<AddmmBackward>)


Before ReLU: tensor([[0.0836, 0.3397, 0.0000, 0.0000, 0.0000, 0.3732, 0.0000, 1.0119, 0.0000,
         0.0000, 0.0000, 0.4261, 0.4022, 0.0000, 0.0000, 0.0669, 0.1184, 0.0000,
         0.1072, 0.2840],
        [0.2232, 0.4901, 0.0000, 0.0000, 0.1000, 0.2607, 0.0000, 1.2661, 0.0000,
         0.0000, 0.0000, 0.6580, 0.0971, 0.0167, 0.00

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [19]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0247, -0.0341,  0.0077,  ...,  0.0198,  0.0314, -0.0232],
        [-0.0246, -0.0255,  0.0174,  ...,  0.0269,  0.0251, -0.0339]],
       device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0055,  0.0301], device='cuda:0', grad_fn=<SliceBackward>)

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0299,  0.0056, -0.0178,  ..., -0.0163, -0.0222,  0.0293],
        [-0.0012,  0.0384, -0.0417,  ..., -0.0100,  0.0359,  0.0169]],
       device='cuda:0', 